## Carregamento e processamento dos Dados

In [17]:
import pandas as pd # manipulação de dados
import numpy as np # operações numéricas, especialmente no cálculo de KPIs e limpeza
# Importação para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Garante que todas as colunas sejam exibidas
pd.set_option('display.max_columns', None) 


In [18]:
# Carregamento dos 5 CSVs
df_orders = pd.read_csv('../data/FACT_Orders.csv', index_col= 0, encoding = 'utf-8')
df_deliverys = pd.read_csv('../data/DIM_Delivery.csv',index_col= 0, encoding = 'utf-8')
df_products = pd.read_csv('../data/DIM_Products.csv',index_col= 0, encoding = 'utf-8')
df_shopping = pd.read_csv('../data/DIM_Shopping.csv',index_col= 0, encoding = 'utf-8')
df_customer = pd.read_csv('../data/DIM_Customer.csv',index_col= 0, encoding = 'utf-8')


In [19]:
# testes de carregamento:

print("Carregamento bem-sucedido. Primeira inspeção:")
print(df_orders.head())

print(df_deliverys.head())

Carregamento bem-sucedido. Primeira inspeção:
             Order_Date  Discount  Subtotal   Total  payment Purchase_Status
Id                                                                          
1   2025-04-12 00:52:33      0.10   3958.20 3586.28  Credito     Processando
2   2025-05-05 01:02:38      0.11   1799.01 1636.64      PIX      Em Analise
3   2025-04-27 23:05:38      0.07   2144.00 2019.82   Boleto       Cancelado
4   2025-04-10 02:42:52      0.03    199.60  237.23   Debito     Processando
5   2025-05-12 21:55:33      0.07    161.70  193.33  Credito      Em Analise
   Delivery_Id  Services  P_Sevice           D_Forecast               D_Date  \
Id                                                                             
1       D00001  Same-Day     42.90  2025-05-03 14:45:17  2025-05-03 14:45:17   
2       D00002  Same-Day     42.90  2025-05-14 07:31:07  2025-05-17 07:31:07   
3       D00003  Standard     22.90  2025-05-10 15:29:43  2025-05-10 15:29:43   
4       D00004 